In [1]:
print("....... chapter 6 yayayaayay")

....... chapter 6 yayayaayay


In [2]:
import urllib.request
import zipfile
import os 
from pathlib import Path

url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"

zip_path = "sms_path_collection.zip"
extracted_path = "sms_spam_collection"

data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

def downlaod_and_unzip_spam_data(
        url , zip_path , extracted_path , data_file_path):
    
    if data_file_path.exists():
        print(f"{data_file_path} already exists . skipping download and extraction")
        return
    with urllib.request.urlopen(url) as resposne:
        with open(zip_path , 'wb') as out_file:
            out_file.write(resposne.read())
    with zipfile.ZipFile(zip_path ,'r') as zip_ref:
        zip_ref.extractall(extracted_path)
    original_file_path  = Path(extracted_path) / 'SMSSpamCollection'
    os.rename(original_file_path , data_file_path)
    print(f'File downloaded and saved as {data_file_path}')            


In [3]:
downlaod_and_unzip_spam_data(url, 
                             zip_path,
                             extracted_path,
                             data_file_path)

sms_spam_collection/SMSSpamCollection.tsv already exists . skipping download and extraction


In [4]:
import pandas as pd
df = pd.read_csv(
    data_file_path,
    sep='\t',
    header=None,
    names=['Label', 'text']
)
df

,Label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
print(df['Label'].value_counts())

Label
ham     4825
spam     747
Name: count, dtype: int64


In [6]:
def create_balanced_dataset(df):
    num_spam = df[df['Label'] == "spam"].shape[0]
    ham_subset = df[df["Label"] == 'ham'].sample(
        num_spam , random_state = 123,
    )
    balanced_df = pd.concat([
        ham_subset , df[df['Label']=='spam']
    ])

    return balanced_df

balanced_df = create_balanced_dataset(df)


In [7]:
balanced_df

,Label,text
4307,ham,Awww dat is sweet! We can think of something t...
4138,ham,Just got to &lt;#&gt;
4831,ham,"The word ""Checkmate"" in chess comes from the P..."
4461,ham,This is wishing you a great day. Moji told me ...
5440,ham,Thank you. do you generally date the brothas?
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [8]:
balanced_df['Label'].value_counts()

Label
ham     747
spam    747
Name: count, dtype: int64

In [9]:
balanced_df['Label']=balanced_df['Label'].map({'spam':0 , "ham":1})

In [10]:
balanced_df

,Label,text
4307,1,Awww dat is sweet! We can think of something t...
4138,1,Just got to &lt;#&gt;
4831,1,"The word ""Checkmate"" in chess comes from the P..."
4461,1,This is wishing you a great day. Moji told me ...
5440,1,Thank you. do you generally date the brothas?
...,...,...
5537,0,Want explicit SEX in 30 secs? Ring 02073162414...
5540,0,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,0,Had your contract mobile 11 Mnths? Latest Moto...
5566,0,REMINDER FROM O2: To get 2.50 pounds free call...


In [11]:
def random_split(df, train_fac, val_fac):
    df = df.sample(frac = 1,random_state=123).reset_index(drop=True)

    train_end = int(len(df) * train_fac)
    val_end = train_end + int(len(df) * val_fac)


    train_df = df[:train_end]
    validation_df = df[train_end:val_end]
    test_df = df[val_end:]

    return train_df,validation_df,test_df

train_df,val_df,test_df = random_split(balanced_df,0.7,0.1)


In [12]:
train_df.to_csv('train.csv',index=None)
val_df.to_csv('val.csv',index=None)
test_df.to_csv('test.csv',index=None)

In [13]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
tokenizer.encode("<|endoftext|>",allowed_special={"<|endoftext|>"})

[50256]

In [14]:
import torch 
from torch.utils.data import Dataset
import pandas as pd  # Added missing import

class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        super().__init__()
        self.data = pd.read_csv(csv_file)

        self.encoded_texts = [tokenizer.encode(text) for text in self.data['text']]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
        
        # Apply truncation first
        self.encoded_texts = [
            encoded_text[:self.max_length] for encoded_text in self.encoded_texts
        ]    

        # Apply padding
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]    

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        ) 

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length


In [15]:
train_dataset = SpamDataset(
    csv_file='train.csv',
    tokenizer=tokenizer,
    max_length=None
)

In [16]:
print(train_dataset.max_length)

120


In [17]:
val_dataset = SpamDataset(
    csv_file='val.csv',
    tokenizer=tokenizer,
    max_length=train_dataset.max_length
)
test_dataset = SpamDataset(
    csv_file='test.csv',
    tokenizer=tokenizer,
    max_length=train_dataset.max_length
)

In [18]:
padded_train_dataset = SpamDataset(
    csv_file='train.csv',
    tokenizer=tokenizer,
    max_length=1024
)
padded_val_dataset = SpamDataset(
    csv_file='val.csv',
    tokenizer=tokenizer,
    max_length=train_dataset.max_length
)
padded_test_dataset = SpamDataset(
    csv_file='test.csv',
    tokenizer=tokenizer,
    max_length=train_dataset.max_length
)

In [23]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8
torch.manual_seed(123)

train_dataloader = DataLoader(
    num_workers = 0,
    batch_size = batch_size,
    shuffle = True,
    dataset=train_dataset,
    drop_last = True
)

val_dataloader = DataLoader(
    num_workers = 0,
    batch_size = batch_size,
    shuffle = True,
    dataset=val_dataset,
    drop_last = True
)

test_dataloader = DataLoader(
    num_workers = 0,
    batch_size = batch_size,
    shuffle = True,
    dataset=test_dataset,
    drop_last = True
)

In [30]:
for input_batch , target_batch in train_dataloader:
    pass

print("Input batch size:",input_batch.shape)
print("target batch size:",target_batch.shape)

Input batch size: torch.Size([8, 120])
target batch size: torch.Size([8])


In [32]:
print("training set length:",len(train_dataloader))
print("val set length:",len(val_dataloader))
print("test set length:",len(test_dataloader))

training set length: 130
val set length: 18
test set length: 37
